In [11]:
import pandas as pd

df = pd.read_csv("../DATA/client_full.csv")
df.head()


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen,last_purchase_date,days_since_last_purchase,churn,frequency,monetary
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19,2024-12-16 21:01:32,15.0,0,44,2306.18
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09,2024-12-26 10:58:03,5.0,0,122,7236.93
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03,2024-12-29 22:46:06,2.0,0,37,2076.58
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11,2024-11-19 02:28:56,42.0,0,13,614.39
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21,NaN,999.0,1,0,0.00


In [12]:
import mlflow
import os

# Ruta absoluta a tu carpeta de trabajo
tracking_path = r"C:\Users\59899\OneDrive\semestre 4\GGD 2\mlruns"

mlflow.set_tracking_uri(f"file:///{tracking_path.replace(os.sep, '/')}")
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///C:/Users/59899/OneDrive/semestre 4/GGD 2/mlruns


In [13]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Seleccionar features NUMÉRICAS
X = df[["age", "days_since_last_purchase", "frequency", "monetary"]]
y = df["churn"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Crear o setear experimento
mlflow.set_experiment("churn_prediction_experiment")


2025/11/14 11:25:23 INFO mlflow.tracking.fluent: Experiment with name 'churn_prediction_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/59899/OneDrive/semestre 4/GGD 2/mlruns/705922343718493732', creation_time=1763130323274, experiment_id='705922343718493732', last_update_time=1763130323274, lifecycle_stage='active', name='churn_prediction_experiment', tags={}>

In [14]:
def mlflow_run(model, model_name):
    with mlflow.start_run(run_name=model_name):

        # Entrenar
        model.fit(X_train, y_train)

        # Predicciones
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        # Métricas
        auc = roc_auc_score(y_test, y_proba)
        f1 = f1_score(y_test, y_pred)

        # Loguear métricas
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("f1", f1)

        # Loguear parámetros
        if hasattr(model, "get_params"):
            mlflow.log_params(model.get_params())

        # Loguear modelo
        mlflow.sklearn.log_model(model, model_name)

        print(f"{model_name} → AUC={auc:.3f} | F1={f1:.3f}")


In [15]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Decision Tree": DecisionTreeClassifier(max_depth=6),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    mlflow_run(model, name)


2025/11/14 11:25:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/14 11:25:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/14 11:25:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logistic Regression → AUC=1.000 | F1=1.000


2025/11/14 11:25:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree → AUC=1.000 | F1=1.000


2025/11/14 11:25:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/14 11:25:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest → AUC=1.000 | F1=1.000


2025/11/14 11:25:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/14 11:25:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting → AUC=1.000 | F1=1.000


In [16]:
import mlflow
import mlflow.sklearn
import pandas as pd

# Cargar dataset final
df = pd.read_csv("../DATA/client_full.csv")

# 1) Columnas numéricas que sí queremos usar
num_cols = ["age", "days_since_last_purchase", "frequency", "monetary"]

# 2) Columnas categóricas a one-hot encodear
cat_cols = ["gender", "city", "country", "citizenship", "document_type"]

# 3) Crear matriz X solo con variables numéricas + dummies
X_num = df[num_cols]
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)

X = pd.concat([X_num, X_cat], axis=1)

# 4) Variable objetivo
y = df["churn"]

# 5) Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6) Entrenar Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

rf.fit(X_train, y_train)

# 7) Predicciones
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# 8) Métricas
test_auc = roc_auc_score(y_test, y_proba)
test_f1 = f1_score(y_test, y_pred)

# 9) MLflow: experimento + run
mlflow.set_experiment("mlflow-churn")

with mlflow.start_run():

    # Log parámetros
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("max_depth", 10)

    # Log métricas
    mlflow.log_metric("test_auc", test_auc)
    mlflow.log_metric("test_f1", test_f1)

    # Log modelo
    mlflow.sklearn.log_model(rf, "random_forest_model")

print("Modelo logueado correctamente 🎯")


2025/11/14 11:25:52 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-churn' does not exist. Creating a new experiment.
2025/11/14 11:25:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/14 11:25:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo logueado correctamente 🎯
